# Tech Intro

## Main tables

In [2]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [3]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = real_estate_list[0] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [4]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

22/11/10 04:38:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/10 04:38:49 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/11/10 04:38:50 WARN Utils: Service 'SparkUI' could not bind on port 4080. Attempting port 4081.
22/11/10 04:38:50 WARN Utils: Service 'SparkUI' could not bind on port 4081. Attempting port 4082.
22/11/10 04:38:50 WARN Utils: Service 'SparkUI' could not bind on port 4082. Attempting port 4083.
22/11/10 04:38:50 WARN Utils: Service 'SparkUI' could not bind on port 4083. Attempting port 4084.
22/11/10 04:38:50 WARN Utils: Service 'SparkUI' could 

## Read the spec-file from Hadoop and show some data sample

In [5]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+------+------------------------------------+--------+--------+------------------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Show columns' properties

In [6]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- lead_id: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- cad_number: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- district: string (nullable = true)
 |-- street: string (nullable = true)
 |-- address: string (nullable = true)
 |-- address_view: string (nullable = true)
 |-- location: string (nullable = true)
 |-- location2: string (nullable = true)
 |-- price: string (nullable = true)
 |-- price_per_meter: string (nullable = true)
 |-- is_new: string (nullable = true)
 |-- rental_duration: string (nullable = true)
 |-- tenant: string (nullable = true)
 |-- object_type: string (nullable = true)
 |-- object_sub_type: string (nullable = true)
 |-- action_type: string (nullable = true)
 |-- rooms: str

## Total rows (count)

In [7]:
eda_df.count()

301099

## Simple func to execute and show the result of the SQL-instruction

In [27]:
def execute(select_instruction: str = "select count(*) from realEstate") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from realEstate"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [9]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [10]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|  301099|
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [11]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,uuid,lead_id,agent_id,cad_number,title,description,country,region,city,...,is_apartment,floor,total_floors,walls_material,empty_fields_count,upload_errors_count,company_id,market_price,pdf_link_hash,agent_bonus
0,0,0,0,298827,208321,0,0,16586,16827,19996,...,0,90828,63025,153549,0,0,0,296247,0,298904


In [12]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,uuid,lead_id,agent_id,cad_number,title,description,country,region,city,...,is_apartment,floor,total_floors,walls_material,empty_fields_count,upload_errors_count,company_id,market_price,pdf_link_hash,agent_bonus
0,0,0,0,0,31,1,0,0,0,0,...,298727,3994,4386,0,234398,278036,0,158,0,415


In [13]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,uuid,lead_id,agent_id,cad_number,title,description,country,region,city,...,is_apartment,floor,total_floors,walls_material,empty_fields_count,upload_errors_count,company_id,market_price,pdf_link_hash,agent_bonus
0,0,0,0,0,4502,300978,58189,383,433,1216,...,0,0,0,0,0,0,0,0,299921,0


In [14]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
uuid,0,0,0
lead_id,0,0,0
agent_id,298827,0,0
cad_number,208321,31,4502
title,0,1,300978
description,0,0,58189
country,16586,0,383
region,16827,0,433
city,19996,0,1216


In [15]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [16]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
uuid,0.000,0.000,0.000
lead_id,0.000,0.000,0.000
agent_id,99.245,0.000,0.000
cad_number,69.187,0.010,1.495
title,0.000,0.000,99.960
description,0.000,0.000,19.326
country,5.508,0.000,0.127
region,5.589,0.000,0.144
city,6.641,0.000,0.404


### SQL-instructions

In [33]:
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|301099            |
+------------------+

+--------------------+
|count(DISTINCT uuid)|
+--------------------+
|301099              |
+--------------------+

+-----------------------+
|count(DISTINCT lead_id)|
+-----------------------+
|301025                 |
+-----------------------+

+------------------------+
|count(DISTINCT agent_id)|
+------------------------+
|216                     |
+------------------------+

+--------------------------+
|count(DISTINCT cad_number)|
+--------------------------+
|80186                     |
+--------------------------+

+---------------------+
|count(DISTINCT title)|
+---------------------+
|120                  |
+---------------------+



+---------------------------+
|count(DISTINCT description)|
+---------------------------+
|233413                     |
+---------------------------+

+-----------------------+
|count(DISTINCT country)|
+-----------------------+
|29                     |
+-----------------------+

+----------------------+
|count(DISTINCT region)|
+----------------------+
|187                   |
+----------------------+

+--------------------+
|count(DISTINCT city)|
+--------------------+
|8956                |
+--------------------+

+------------------------+
|count(DISTINCT district)|
+------------------------+
|4782                    |
+------------------------+

+----------------------+
|count(DISTINCT street)|
+----------------------+
|15167                 |
+----------------------+

+-----------------------+
|count(DISTINCT address)|
+-----------------------+
|155175                 |
+-----------------------+

+----------------------------+
|count(DISTINCT address_view)|
+--------------------

In [141]:
#Find dupes or misunderstanding or useless static values 301 099 total

execute("SELECT lead_id,\
        count(lead_id) AS quantity \
        FROM eda_sql\
        GROUP BY lead_id\
        HAVING count(lead_id) > 1 \
        ORDER BY quantity DESC")

execute("SELECT CAST(id AS integer),\
        uuid \
        FROM eda_sql\
        ORDER BY id DESC, uuid DESC")

execute("SELECT cad_number,\
        count(cad_number) AS quantity \
        FROM eda_sql\
        GROUP BY cad_number\
        HAVING count(cad_number) > 1 \
        ORDER BY quantity DESC")

execute("SELECT \
        count(description) AS quantity \
        FROM eda_sql\
        GROUP BY description\
        HAVING count(description) > 1 \
        ORDER BY quantity DESC")

execute("SELECT DISTINCT(country)\
        FROM eda_sql")

execute("SELECT DISTINCT location, location2\
        FROM eda_sql")

execute("SELECT price_per_meter,\
        count(price_per_meter) AS quantity \
        FROM eda_sql\
        GROUP BY price_per_meter\
        ORDER BY price_per_meter DESC")

execute("SELECT distinct(cast(rooms as integer))\
        FROM eda_sql\
        ORDER BY ROOMS DESC")

execute("SELECT distinct(video_url)\
        FROM eda_sql\
        ORDER BY video_url desc")

execute("SELECT distinct(cast(filial_id as integer))\
        FROM eda_sql\
        ORDER BY filial_id desc")

execute("SELECT distinct(house)\
        FROM eda_sql\
        ORDER BY house desc")

execute("SELECT distinct(apartment)\
        FROM eda_sql\
        ORDER BY apartment desc")

execute("SELECT distinct(cad_number_land)\
        FROM eda_sql\
        ORDER BY cad_number_land desc")

execute("SELECT distinct(company_id)\
        FROM eda_sql\
        ORDER BY company_id desc")

execute("SELECT distinct(cast(market_price as integer))\
        FROM eda_sql\
        ORDER BY market_price asc")

execute("SELECT distinct(pdf_link_hash)\
        FROM eda_sql\
        ORDER BY pdf_link_hash ASC")

execute("SELECT distinct(address_view)\
        FROM eda_sql\
        ORDER BY address_view ASC")

execute("SELECT distinct(area)\
        FROM eda_sql\
        ORDER BY area DESC")

execute("SELECT distinct(agent_bonus)\
        FROM eda_sql\
        ORDER BY agent_bonus ASC")

+--------+--------+
|lead_id |quantity|
+--------+--------+
|21311113|15      |
|21311869|5       |
|21398176|3       |
|21311825|3       |
|21302510|3       |
|21297209|3       |
|21679515|3       |
|21311639|3       |
|21779040|2       |
|21398168|2       |
|21308048|2       |
|21312270|2       |
|21339294|2       |
|21427558|2       |
|21337573|2       |
|21298091|2       |
|21312349|2       |
|21297192|2       |
|21310367|2       |
|21393464|2       |
+--------+--------+
only showing top 20 rows



+------+------------------------------------+
|id    |uuid                                |
+------+------------------------------------+
|333129|3adda455-f6a1-3f20-43f0-5deb40fccffb|
|333128|71a5bac3-3733-f012-96e8-258ca5fed78e|
|333127|afc0b0f1-591c-f74a-221b-d96cf92fbab0|
|333126|6a0392d3-88ea-ed34-afe1-f84906d0edb9|
|333125|638d28a3-fbf7-9552-e3aa-d5f8cb3f91a6|
|333124|cc0c7edf-6ab4-32e7-4f43-5a341caaeb42|
|333123|fb51be5b-e204-c708-431a-c8c0a5db2d80|
|333122|447823e6-2987-6d4a-90c6-94498f1d239e|
|333121|f1e138eb-7c20-cfe9-eb1b-91311ba45bc9|
|333120|948e6a68-3f9d-bcdd-92f9-c79e2ec5b2f4|
|333119|5382ca67-0c95-0ceb-f611-6aff87e10c4c|
|333118|8ad0ceb8-6f84-4e91-863a-6e84dd3f1217|
|333117|871cd5f0-c009-89df-7cba-71a7edcdf09e|
|333116|1780f0a4-4ae3-95f7-438d-e52b167ca7c0|
|333115|abf10147-fdf0-a320-68ad-0f919ae6275f|
|333114|5ddc75ea-4b2d-a5d7-d4c2-f9edf6f0bb2b|
|333113|c14f2355-f5c9-e515-5111-291df8ad6c1b|
|333112|277b8de8-cccb-bf3d-5d35-f75a8ce57f4e|
|333111|b5906822-c054-251d-43d2-66

+------------------------------+--------+
|cad_number                    |quantity|
+------------------------------+--------+
|                              |4502    |
|1                             |84      |
|-                             |49      |
|02:55:020631:1803             |37      |
|02:55:050504:2277             |27      |
|24:50:0000000:193107          |27      |
|02:55:030267:1455             |20      |
|02:55:020603:5236             |20      |
|02:55:010131:686              |18      |
|22:65:000000:0000:01:405:002:0|18      |
|02:55:050229:528              |17      |
|Республика Башкортостан       |17      |
|0                             |15      |
|02:55:020412:2801             |15      |
|8052:5789:2341:8974           |15      |
|02:55:050229:529              |15      |
|02:55:050702:97               |14      |
|02:55:010225:339              |14      |
|Белгородская область          |13      |
|Ярославская область           |12      |
+------------------------------+--

+--------+
|quantity|
+--------+
|58189   |
|712     |
|646     |
|109     |
|102     |
|87      |
|82      |
|81      |
|78      |
|78      |
|75      |
|65      |
|63      |
|60      |
|56      |
|55      |
|48      |
|47      |
|45      |
|45      |
+--------+
only showing top 20 rows



+-----------------------------+
|country                      |
+-----------------------------+
|Беларусь                     |
|null                         |
|Узбекистан                   |
|Абхазия                      |
|Россия                       |
|россия                       |
|Объединенные Арабские Эмираты|
|РФ                           |
|Армения                      |
|РОс                          |
|Чехия                        |
|Азербайджан                  |
|Казахстан                    |
|Казахстан                    |
| РоссияоРОРоссия             |
|                             |
|Россия                       |
|Таиланд                      |
|Турция                       |
|Киргизия                     |
+-----------------------------+
only showing top 20 rows

+--------------------------------------------------+------------------------------------------------------------------------------------------+
|location                                          |location2 

+---------------+--------+
|price_per_meter|quantity|
+---------------+--------+
|9999999999999  |1       |
|99999997952    |1       |
|99900          |105     |
|999            |1       |
|99800          |124     |
|998            |2       |
|99700          |143     |
|997            |1       |
|99600          |93      |
|99500          |125     |
|995            |1       |
|99400          |125     |
|994            |1       |
|99300          |112     |
|99200          |97      |
|992            |2       |
|99100          |129     |
|99000          |167     |
|9900           |21      |
|990            |1       |
+---------------+--------+
only showing top 20 rows

+-----+
|rooms|
+-----+
|255  |
|229  |
|220  |
|210  |
|190  |
|168  |
|167  |
|150  |
|140  |
|138  |
|123  |
|114  |
|113  |
|112  |
|111  |
|110  |
|109  |
|100  |
|93   |
|92   |
+-----+
only showing top 20 rows

+----------------------------------------------------+
|video_url                                           

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [142]:
spark.stop()

I1110 07:32:53.662405 29725 sched.cpp:2174] Asked to stop the driver
I1110 07:32:53.662559 30188 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-56916
